# **Filogenia**

Neste Notebook, iremos realizar algumas etapas importantes e necessárias para podermos reconstruir uma hipótese filogenética sobre um vírus zoonótico amplamente prevalente no Brasil, o vírus da Dengue. Para isso, realizaremos a reconstrução filogenética utilizando o método de máxima verossimilhança:
*   Alinhamento dados de sequências genômicas;
*   Avaliar sinal filogenético;
*   Escolher o melhor modelo de substituição de nucleotídeos;
*   Gerar a árvore filogenética.

## Programas ou pacotes necessários

*   **MAFFT** para alinhamento do genoma (https://mafft.cbrc.jp/alignment/software/)


*   **IQ-TREE** para construção de árvore filogenética de máxima verossimilhança (http://www.iqtree.org/)


*   **ETETOOLKIT** para visualização e manipulação de árvores filogenéticas (http://etetoolkit.org/)


*   **iTOL** para visualização e edição da árvore filogenética (https://itol.embl.de/).


## Outros programas interessantes


*   **Figtree** um programa em Java para visualizar as árvores filogenéticas (http://tree.bio.ed.ac.uk/software/figtree/).


*   **ggtree** um programa em R para visualizar as árvores filogenéticas (https://github.com/YuLab-SMU/ggtree , https://guangchuangyu.github.io/software/ggtree/).

*   **Dendroscope** um programa em Java para visualizar as árvores filogenéticas (https://uni-tuebingen.de/fakultaeten/mathematisch-naturwissenschaftliche-fakultaet/fachbereiche/informatik/lehrstuehle/algorithms-in-bioinformatics/software/dendroscope/).




Vamos primeiro checar a versão do python

In [ ]:
!python --version

Python 3.10.12


Instalando o conda/mamba

In [ ]:
!pip install --upgrade --force-reinstall zstandard cuda-python
!pip install -q condacolab

!sed -i '/cudatoolkit/d' /usr/local/conda-meta/pinned

#Código em Python
import condacolab
condacolab.install()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 49.7 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.22.0
    Uninstalling zstandard-0.22.0:
      Successfully uninstalled zstandard-0.22.0
✨🍰✨ Everything looks OK!


In [ ]:
#!conda install -c bioconda mafft iqtree
!mamba install -c bioconda mafft iqtree --quiet

## Baixando o conjunto de dados a ser analisado.

Utilizaremos um conjunto de dados de DENV para esta aula prática. O conjunto possui um isolado chamado “ID9999|BR-Bahia|2016” e o objetivo é determinar o tipo e genótipo de DENV que circulou na Bahia em 2016.

In [ ]:
!wget https://raw.githubusercontent.com/khourious/IGM-TRAIN/main/course_data/IGM-TRAIN_DENV.fasta

In [ ]:
#Checando o arquivo
!head IGM-TRAIN_DENV.fasta

#Número de sequencias
!head IGM-TRAIN_DENV.fasta | grep -c '>'

#E agora?
!grep -c '>' IGM-TRAIN_DENV.fasta

### Alinhamento

Usaremos MAFFT para alinhamento das sequências. É bastante rápido e preciso.

Existem muitas opções para alinhar sequências utilizando o MAFFT. --auto é uma boa opção onde o próprio programa escolhe o algoritmo mais eficiente (bom equilíbrio entre velocidade e precisão). O alinhamento pode levar dias para sequências muito longas, se tiver optado pelo algoritmo mais preciso. Se você estiver alinhando sequências curtas, como um gene menos conservado, poderá usar opções mais custosas computacionalmente mas também mais precisas. O MAFFT tem bons exemplos do que usar e quando usar em seu site: https://mafft.cbrc.jp/alignment/software/algorithms/algorithms.html.

* Dica: Quando você estiver trabalhando com alinhamentos contendo centenas ou milhares de sequências de um mesmo vírus o mafft pode ser utilizado para alinhar rapidamente sequencias novas a um alinhamento já existente a partir do uso do mafft --add. Esse parâmetro e suas varações permite alinhar genomas completos bem como fragmentos ([ver documentação aqui](https://mafft.cbrc.jp/alignment/software/addsequences.html))

**EMBOSS**


*   water	- Smith-Waterman local alignment.
*   needle -
*   dottup -

In [ ]:
!mafft --help

In [ ]:
!mafft --auto IGM-TRAIN_DENV.fasta > IGM-TRAIN_DENV_aln.fasta

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 12 ambiguous characters.
    1 / 35
done.

Constructing a UPGMA tree (efffree=0) ... 
   30 / 35
done.

Progressive alignment 1/2... 
STEP    34 / 34  f
done.

Making a distance matrix from msa.. 
    0 / 35
done.

Constructing a UPGMA tree (efffree=1) ... 
   30 / 35
done.

Progressive alignment 2/2... 
STEP    34 / 34  f
done.

disttbfast (nuc) Version 7.525
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert m

In [ ]:
!cat IGM-TRAIN_DENV_aln.fasta

>ID9999|BR-Bahia|2016
-------agtctgtgtggaccgacaaggacagttccaaatcggaagct-tgcttaacaca
gttctaacagttta-tttgaatagagagcagatctctgg-aaaaatgaaccaacgaaaaa
aggtggttagaccacctttcaatatgctgaaacgcgagagaaaccgcgtatcaacccctc
aagggttggtgaagagattctcaaccggacttttttctgggaaaggacccttacggatgg
tgctagcactcatcacgtttttgcgagtcctttccatcccaccaacagcagggattctga
agagatggggacagttgaagaaaaataaggccatcaagatactgattggattcaggaagg
agataggccgcatgctgaacatcttgaacgggagaaaaaggtcaacgataacattgttgt
gcttgattcccaccgtaatggcgtttcacttgtcaacaagagatggcgaacccctcatga
tagtggcaaaacatgaaagggggagacctctcttgtttaagacaacagaggggatcaaca
aatgcactctcattgccatggacttgggtgaaatgtgtgaggacactgtcacgtacaaat
gccccctactggtcaataccgaacctgaagacattgattgctggtgcaacctcacgtcta
cctgggtcatgtatgggacatgcacccagagcggagaacggagacgagagaagcgctcag
tagctttaacaccacattcaggaatgggattggaaacaagagctgagacatggatgtcat
cggaaggagcttggaagcatgctcagagagtagagagctggatactcagaaacccaggat
ttgcactcttggcaggatttatggcttatatgattgggcaaacagggatccagcgaactg
tcttctttgtcctaatgatgctggtcgccccatcctacggaatgcgatgcgtaggagtag
ga

### Instalando a ferramenta para visualizar o alinhamento

Primeiro a biblioteca biopython e depois o programa de visualização (alv)

In [ ]:
pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.4 MB/s eta 0:00:00


In [ ]:
pip install alv

In [ ]:
from Bio import AlignIO
msa = AlignIO.read('IGM-TRAIN_DENV_aln.fasta', 'fasta', )
import alv
alv.view(msa, width = 80)

TypeError: get_alv_objects() missing 2 required positional arguments: 'al_start' and 'al_end'

In [ ]:
from Bio import AlignIO
msa = AlignIO.read('IGM-TRAIN_DENV_aln.fasta', 'fasta',   )
import alv
alv.view(msa)

### Inferência Filogenética

O IQ-TREE permite realizar análises filogenéticas de máxima verossimilhança de centenas ou milhares de sequências. Com ele é possível:

*   **Avaliar a informação filogenética de um alinhamento:** recomenda-se que o número de quartetos avaliados sejam pelo menos 25 vezes o número de sequências no alinhamento, garantindo que cada sequência seja coberta aproximadamente 100 vezes no conjunto de quartetos sorteados na análise. *O alinhamento **IGM-TRAIN_DENV_aln.fasta** possui 35 sequências, ou seja, a análise irá avaliar pelo menos 875 quartetos.*
*   **Estimar o modelo evolutivo:** possui um programa integrado chamado ModeFinder, o qual avalia rapidamente uma grande quantidade de modelos de substituição de nucleotídeos e escolhe o mais adequado para o conjunto de dados em questão.
*   **Inferir árvore de máxima verossimilhança:** estima e avalia a robustez da melhor árvore utilizando abordagens como bootstrap, bootstrap ultra-rápido ou avaliações probabilísticas.

O programa também pode ser executado em servidores online:
*   W-IQ-TREE Server – http://iqtree.cibiv.univie.ac.at/
*   Cyberinfrastructure for Phylogenetic Research (CIPRES) – https://www.phylo.org/
*   HIV Sequence Database – https://www.hiv.lanl.gov/content/sequence/IQTREE/iqtree.html

In [ ]:
!iqtree -h

IQ-TREE multicore version 2.3.0 COVID-edition for Linux 64-bit built Mar 14 2024
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung,
Olga Chernomor, Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan.

Usage: iqtree [-s ALIGNMENT] [-p PARTITION] [-m MODEL] [-t TREE] ...

GENERAL OPTIONS:
  -h, --help           Print (more) help usages
  -s FILE[,...,FILE]   PHYLIP/FASTA/NEXUS/CLUSTAL/MSF alignment file(s)
  -s DIR               Directory of alignment files
  --seqtype STRING     BIN, DNA, AA, NT2AA, CODON, MORPH (default: auto-detect)
  -t FILE|PARS|RAND    Starting tree (default: 99 parsimony and BIONJ)
  -o TAX[,...,TAX]     Outgroup taxon (list) for writing .treefile
  --prefix STRING      Prefix for all output files (default: aln/partition)
  --seed NUM           Random seed number, normally used for debugging purpose
  --safe               Safe likelihood kernel to avoid numerical underflow
  --mem NUM[G|M|%]     Maximal RAM usage in GB | MB | %
  --runs NU

In [ ]:
!iqtree -s IGM-TRAIN_DENV_aln.fasta -lmap 875 -m TEST --alrt 1000 -B 1000
# -s: nome do arquivo de alinhamento
# -lmap: número de quartetos a serem sorteados aleatoriamente. Se especificar -lmap ALL, todos os quartetos únicos serão sorteados
# -m: estratégia de seleção de modelo. Se nenhuma opção for especificada
# -alrt: método para avaliar o suporte dos clados utilizando approximate likelihood ratio test, número de réplicas - resultados podem ser recuperados na .treefile
# -B: método alternativa para avaliar o suporte dos clados, número de réplicas para bootstrap ultra-rápido - resultados podem ser recuperados na .treefile e na .contree

IQ-TREE multicore version 2.3.0 COVID-edition for Linux 64-bit built Mar 14 2024
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung,
Olga Chernomor, Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan.

Host:    d0ba42b1a56a (AVX2, FMA3, 12 GB RAM)
Command: iqtree -s IGM-TRAIN_DENV_aln.fasta -lmap 875 -m TEST --alrt 1000 -B 1000
Seed:    659245 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Fri Mar 15 19:28:39 2024
Kernel:  AVX+FMA - 1 threads (2 CPU cores detected)

HINT: Use -nt option to specify number of threads because your CPU has 2 cores!
HINT: -nt AUTO will automatically determine the best number of threads to use.

Reading alignment file IGM-TRAIN_DENV_aln.fasta ... Fasta format detected
Reading fasta file: done in 0.0112075 secs using 77.39% CPU
Alignment most likely contains DNA/RNA sequences
Alignment has 35 sequences with 10800 columns, 3110 distinct patterns
4008 parsimony-informative, 1620 singleton sites, 5172 constant sites
  

Quando o processo estiver concluído, os arquivos de saída serão encontrados na pasta, incluindo:

*   **IGM-TRAIN_DENV_aln.fasta.lmap.svg:** gráfico de mapeamento de verossimilhança (SVG)
*   **IGM-TRAIN_DENV_aln.fasta.treefile:** a árvore ML no formato NEWICK, que pode ser visualizada por qualquer programa de visualização de árvore que suporta NEWICK
*   **IGM-TRAIN_DENV_aln.fasta.contree:** a árvore ML consenso obtida pelo fastbootstrap no formato NEWICK, que pode ser visualizada por qualquer programa de visualização de árvore suportado.
*   **IGM-TRAIN_DENV_aln.fasta.iqtree:** o arquivo principal de relatório da análise. Ele também contém uma representação textual da árvore final
*   **IGM-TRAIN_DENV_aln.fasta.log:** arquivo de log de toda a execução (o mesmo que é exibido na tela).

### Interpretação do sinal filogenético

![IGM-TRAIN_DENV_aln.fasta.lmap[1].svg]()

*   **Triângulo do top:** distribuição dos quartetos representados por pontos.
*   **Triângulo à esquerda:** as três áreas mostram apoio para um dos diferentes agrupamentos formado pelos quartetos (a,b)-(c,d), (a,c)-(b,d) e (a,d)-(b,c).
*   **Triângulo à direita:** quartetos que caem nos três cantos são informativos. Os retângulos são parcialmente informativos e os quartetos no centro são não informativos.

***Um bom conjunto de dados deve ter um alto número de quartetos informativos e um baixo número de quartetos não informativos.***

### Para visualizar o gráfico de sinal filogenético:

1.   Ir na aba à esquerda do Google Colab da figura do diretório/pasta.
Baixar o arquivo **IGM-TRAIN_DENV_aln.fasta.lmap.svg** clicando nos três pontinhos ao lado do nome do arquivo.
2.   Abrir o arquivo **IGM-TRAIN_DENV_aln.fasta.lmap.svg** no navegador.

In [ ]:
#Visualizando as árvores no formato newick
!cat IGM-TRAIN_DENV_aln.fasta.treefile
!cat IGM-TRAIN_DENV_aln.fasta.contree

(ID9999|BR-Bahia|2016:0.0023468687,(((((((((((EF457906|MY|Sylvatic|1975:0.1109554027,((FJ196845|CO|DENV1|2005:0.5702407003,JN697379|BR|DENV3|2006:0.4865436779)100/100:0.3258743366,JN796245|US|DENV2|2008:0.7135058389)100/100:0.9200773441)98.6/97:0.0397013287,(AY618988|TH|DENV4_Gen3|1997:0.0013267995,AY618989|TH|DENV4_Gen3|1997:0.0012292840)100/100:0.0559089115)100/99:0.0157987737,((((AY947539|PH|DENV4_Gen1|1956:0.0003228733,GQ868594|PH|DENV4_Gen1|1956:0.0004671991)98.7/100:0.0021039107,FJ196850|CN|DENV4_Gen1|1990:0.0262677503)100/100:0.0094407736,((AY618991|TH|DENV4_Gen1|1977:0.0035077285,((AY618990|TH|DENV4_Gen1|1991:0.0001373333,JQ513345|BR|DENV4_Gen1|2011:0.0106804068)97.3/100:0.0014987803,AY618992|TH|DENV4_Gen1|2001:0.0088917810)100/100:0.0130453062)100/100:0.0075454235,KF041260|PK|DENV4_Gen1|2009:0.0294481186)100/100:0.0158759800)100/100:0.0111961986,JF262783|IN|DENV4_Gen1|1961:0.0315208325)7.2/54:0.0019384550)100/100:0.0105918285,(FJ196849|CN|DENV4_Gen2|1978:0.0240868031,((KC76269

####Instalação do ETE para visualização e manipulação de arvores

Site oficial do ETE Toolkit (http://etetoolkit.org/).

Os tutoriais oficiais *[Guia de referência e tutoriais oficiais](http://etetoolkit.org/docs/latest/tutorial/index.html).

Um tutotial não oficial pode ser encontrado *[aqui](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://evomics.org/wp-content/uploads/2019/01/ete_tutorial.pdf)

Passos de criação do ambiente e instalação

In [ ]:
!mamba create -n ete3 python=3
!mamba env list
!mamba activate ete3
!mamba install -c etetoolkit ete3 ete_toolchain
!mamba install -y pip six numpy pyqt lxml --quiet
!pip install --upgrade ete3


Looking for: ['python=3']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   1%
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64   7%
conda-forge/noarch    26%[+] 0.4s
conda-forge/linux-64  12%
conda-forge/noarch    38%[+] 0.5s
conda-forge/linux-64  16%
conda-forge/noarch    49%[+] 0.6s
conda-forge/linux-64  21%
conda-forge/noarch    61%[+] 0.7s
conda-forge/linux-64  26%
conda-forge/noarch    61%[+] 0.8s
conda-forge/linux-64  31%
conda-forge/noarch    73%[+] 0.9s
conda-forge/linux-64  31%
conda-forge/noarch    85%[+] 1.0s
conda-forge/linux-64  36%
conda-forge/noarch    85%[+] 1.1s
conda-forge/linux-64  36%
conda-forge/noarch    97%[+] 1.2s
conda-forge/linux-64  41%
conda-forge/noarch   100%conda-forge/noarch                                
[+] 1.3s
conda-forge/linux-64  41%[+] 1.4s
conda-forge/linux-64  46%[+] 1.5s
conda-forge/linux-64  56%[+] 1.6s
conda-forge/linux-64  61%[+] 1.7s
conda-forge/linux-64  66%[+] 1.8s
conda-forge/l

Visualizando uma árvore

In [ ]:
from ete3 import Tree

t = Tree("IGM-TRAIN_DENV_aln.fasta.contree")

print(t)


   /-ID9999|BR-Bahia|2016
  |
  |                              /-EF457906|MY|Sylvatic|1975
  |                             |
  |                           /-|      /-FJ196845|CO|DENV1|2005
  |                          |  |   /-|
  |                          |   \-|   \-JN697379|BR|DENV3|2006
  |                        /-|     |
  |                       |  |      \-JN796245|US|DENV2|2008
  |                       |  |
  |                       |  |   /-AY618988|TH|DENV4_Gen3|1997
  |                       |   \-|
  |                       |      \-AY618989|TH|DENV4_Gen3|1997
  |                       |
  |                       |            /-AY947539|PH|DENV4_Gen1|1956
  |                       |         /-|
  |                     /-|      /-|   \-GQ868594|PH|DENV4_Gen1|1956
  |                    |  |     |  |
  |                    |  |     |   \-FJ196850|CN|DENV4_Gen1|1990
  |                    |  |     |
  |                    |  |   /-|      /-AY618991|TH|DENV4_Gen1|1977
  |  

Realizando o enraizamento correto da árvore.

In [ ]:
from ete3 import PhyloTree

t = PhyloTree("IGM-TRAIN_DENV_aln.fasta.contree", format=2)
root_point = t.get_midpoint_outgroup()
t.set_outgroup(root_point)

print(t)


         /-FJ196845|CO|DENV1|2005
      /-|
   /-|   \-JN697379|BR|DENV3|2006
  |  |
  |   \-JN796245|US|DENV2|2008
  |
  |   /-EF457906|MY|Sylvatic|1975
--|  |
  |  |      /-AY618988|TH|DENV4_Gen3|1997
  |  |   /-|
  |  |  |   \-AY618989|TH|DENV4_Gen3|1997
  |  |  |
   \-|  |               /-AY947539|PH|DENV4_Gen1|1956
     |  |            /-|
     |  |         /-|   \-GQ868594|PH|DENV4_Gen1|1956
     |  |        |  |
     |  |        |   \-FJ196850|CN|DENV4_Gen1|1990
     |  |        |
     |  |      /-|      /-AY618991|TH|DENV4_Gen1|1977
      \-|     |  |     |
        |     |  |   /-|      /-AY618990|TH|DENV4_Gen1|1991
        |     |  |  |  |   /-|
        |     |  |  |   \-|   \-JQ513345|BR|DENV4_Gen1|2011
        |   /-|   \-|     |
        |  |  |     |      \-AY618992|TH|DENV4_Gen1|2001
        |  |  |     |
        |  |  |      \-KF041260|PK|DENV4_Gen1|2009
        |  |  |
        |  |   \-JF262783|IN|DENV4_Gen1|1961
        |  |
         \-|      /-FJ196849|CN|DENV4_Gen2|1

Extraindo o ancestral do nodo de interesse.

In [ ]:
from ete3 import Tree

t = Tree("IGM-TRAIN_DENV_aln.fasta.contree")
root_point = t.get_midpoint_outgroup()
t.set_outgroup(root_point)

ancestor = t.get_common_ancestor("JN983813|BR|DENV4_Gen2|2010", "GQ868585|CO|DENV4_Gen2|2005")

print(ancestor)


      /-GQ868585|CO|DENV4_Gen2|2005
   /-|
  |  |   /-JQ513339|BR|DENV4_Gen2|2011
  |   \-|
--|      \-JQ513344|BR|DENV4_Gen2|2011
  |
  |   /-ID9999|BR-Bahia|2016
   \-|
      \-JN983813|BR|DENV4_Gen2|2010


Imprimindo a árvore enraizada em um novo arquivo:


Formatos disponíveis

0	flexible with support values

1	flexible with internal node names

2	all branches + leaf names + internal supports

3	all branches + all names

4	leaf branches + leaf names

5	internal and leaf branches + leaf names

6	internal branches + leaf names

7	leaf branches + all names

8	all names

9	leaf names

100	topology only

In [ ]:
from ete3 import PhyloTree

t = PhyloTree("IGM-TRAIN_DENV_aln.fasta.contree")
root_point = t.get_midpoint_outgroup()
t.set_outgroup(root_point)

t.write(format=2, outfile="IGM-TRAIN_DENV_aln.fasta.contree.rooted.nwk")

### Perguntas

1.   Qual é o modelo evolutivo mais adequado para este conjunto de dados?
2.   O conjunto de dados possui sinal filogenético?
3.   Qual a porcentagem de quarteros totalmente resolvidos?


### Visualização externa da árvore filogenética

Para isso utilizaremos o iTOL, siga os seguintes passos.

1.   Ir na aba à esquerda do Google Colab da figura do diretório/pasta.
2.   Baixar o arquivo **IGM-TRAIN_DENV_aln.fasta.contree.rooted.nwk** clicando nos três pontinhos ao lado do nome do arquivo.
3.   Ir para o site: https://itol.embl.de/.
4.   Clicar em **Upload** e enviar o arquivo baixado **IGM-TRAIN_DENV_aln.fasta.contree.rooted.nwk**.
5.   Editar a árvore com as seguintes condições:
> * Basic – Mode: Rectangular
> * Basic – Branch lengths: Use
> * Basic – Label options: Position At tips
> * Advanced – Leaf sorting: Default
> * Advanced – Invert sort order: Yes
> * Advanced – Bootstraps: Text, Font 15px
> * Advanced – Root the tree midpoint: Midpoint root


### Perguntas

1.   Qual tipo e genótipo de DENV a sequência “ID9999|BR-Bahia|2016” pertence?
2.   Qual é a sequência mais relacionada (mais próxima) da “ID9999|BR-Bahia|2016”?

Para poder salvar as alterações desse arquivo notebook no seu Drive:
*   Clique em **Arquivo** no canto superior esquerdo e selecione **Salvar uma cópia no Drive**.